In [1]:
from regraph import plot_graph

from kami import KamiCorpus
from kami.data_structures.entities import *
from kami.data_structures.interactions import *
from kami.data_structures.definitions import *

In [2]:
# Create an empty KAMI corpus
corpus = KamiCorpus("EGFR_signalling")

In [3]:
# Create an interaction object
egfr = Protoform("P00533")

egfr_kinase = RegionActor(
    protoform=egfr,
    region=Region(
        name="Protein kinase",
        start=712,
        end=979,
        states=[State("activity", True)]))

interaction1 = LigandModification(
    enzyme=egfr_kinase,
    substrate=egfr,
    target= Residue(
        "Y", 1092,
        state=State("phosphorylation", False)),
    value=True,
    rate=1,
    desc="Phosphorylation of EGFR homodimer")

In [4]:
interaction1.to_json()

{'desc': 'Phosphorylation of EGFR homodimer',
 'enzyme': {'data': {'protoform': {'bound_to': [],
    'regions': [],
    'residues': [],
    'sites': [],
    'states': [],
    'unbound_from': [],
    'uniprotid': 'P00533'},
   'region': {'bound_to': [],
    'end': 979,
    'name': 'Protein kinase',
    'residues': [],
    'sites': [],
    'start': 712,
    'states': [{'name': 'activity', 'test': True}],
    'unbound_from': []}},
  'type': 'RegionActor'},
 'enzyme_bnd_subactor': 'protoform',
 'rate': 1,
 'substrate': {'data': {'bound_to': [],
   'regions': [],
   'residues': [],
   'sites': [],
   'states': [],
   'unbound_from': [],
   'uniprotid': 'P00533'},
  'type': 'Protoform'},
 'substrate_bnd_subactor': 'protoform',
 'target': {'data': {'aa': ['Y'],
   'loc': 1092,
   'state': {'name': 'phosphorylation', 'test': False}},
  'type': 'Residue'},
 'type': 'LigandModification',
 'value': True}

In [5]:
# Aggregate the interaction object to the corpus
nugget1_id = corpus.add_interaction(interaction1)

/home/eugenia/anaconda3/lib/python3.6/site-packages/kami-1.2-py3.6.egg/anatomizer/new_anatomizer.py:283: AnatomizerWarning: Cannot access remote, giving up InterPro update for now.
  AnatomizerWarning)


Loading InterPro Data version 66


In [6]:
# Access the newly created nugget
nugget = corpus.get_nugget(nugget1_id)
print(corpus.get_nugget_desc(nugget1_id))
nugget_identification = corpus.get_nugget_typing(nugget1_id)
print("Nugget nodes: ")
for n in nugget.nodes():
    print("-> '{}' - identified as '{}' in the AG, typed as '{}' in the MM".format(
        n, nugget_identification[n], corpus.get_action_graph_typing()[nugget_identification[n]]))

Phosphorylation of EGFR homodimer
Nugget nodes: 
-> 'P00533' - identified as 'P00533_P00533_1' in the AG, typed as 'protoform' in the MM
-> 'P00533_region_Protein kinase_712_979' - identified as 'P00533_region_Protein kinase_712_979' in the AG, typed as 'region' in the MM
-> 'P00533_region_Protein kinase_712_979_activity' - identified as 'P00533_region_Protein kinase_712_979_activity' in the AG, typed as 'state' in the MM
-> 'P00533_1' - identified as 'P00533_P00533_1' in the AG, typed as 'protoform' in the MM
-> 'mod' - identified as 'mod' in the AG, typed as 'mod' in the MM
-> 'P00533_1_Y1092' - identified as 'P00533_1_Y1092' in the AG, typed as 'residue' in the MM
-> 'P00533_1_Y1092_phosphorylation' - identified as 'P00533_1_Y1092_phosphorylation' in the AG, typed as 'state' in the MM
-> 'is_bnd' - identified as 'is_bnd' in the AG, typed as 'bnd' in the MM


In [7]:
# Manually add a new protoform
new_protoform_node = corpus.add_protoform(Protoform("P62993"))
print(new_protoform_node)

# Access protoforms
corpus.protoforms()

P62993


['P00533_P00533_1', 'P62993']

In [8]:
# Manually add a new components to an arbitrary protoform

corpus.add_site(Site("New site"), new_protoform_node)
print(corpus.get_attached_sites(new_protoform_node))

['P62993_site_New site']


In [9]:
grb2 = Protoform("P62993")
grb2_sh2 = RegionActor(
    protoform=grb2,
    region=Region(name="SH2"))

shc1 = Protoform("P29353")
shc1_pY = SiteActor(
    protoform=shc1,
    site=Site(
        name="pY",
        residues=[Residue("Y", 317, State("phosphorylation", True))]))
interaction1 = Binding(grb2_sh2, shc1_pY)

grb2_sh2_with_residues = RegionActor(
    protoform=grb2,
    region=Region(
        name="SH2",
        residues=[
            Residue("S", 90, test=True),
            Residue("D", 90, test=False)]))

egfr_pY = SiteActor(
    protoform=egfr,
    site=Site(
        name="pY",
        residues=[Residue("Y", 1092, State("phosphorylation", True))]))

interaction2 = Binding(grb2_sh2_with_residues, shc1_pY)

In [10]:
corpus.add_interactions([interaction1, interaction2])

['EGFR_signalling_nugget_2', 'EGFR_signalling_nugget_3']

In [11]:
# Create a protein definition for GRB2
protoform = Protoform(
    "P62993",
    regions=[Region(
        name="SH2",
        residues=[
            Residue("S", 90, test=True),
            Residue("D", 90, test=False)])])

ashl = Product(residues=[Residue("S", 90)])
s90d = Product(residues=[Residue("D", 90)])
grb3 = Product(removed_components={"regions": [Region("SH2")]})

grb2_definition = Definition(protoform, [ashl, s90d, grb3])

In [14]:
grb2_definition.generate_rule(corpus.action_graph, corpus.get_action_graph_typing())

KeyError: 'P62993_region_SH2_D90_P62993_region_SH2_S90'